<a href="https://colab.research.google.com/github/namratabhattacharya/ROclassifier/blob/main/BoostedClassifier_ranklogFC_expr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Rank ordered logFC*expression on Pancreas training Dataset

In [ ]:
!pip install scanpy -q

In [ ]:
import numpy as np
import pandas as  pd
import scanpy as sc
from sklearn.linear_model import LinearRegression


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# TRAINING DATA

## Read the Training data

In [ ]:
## Read train data
adata_train = sc.read('/content/gdrive/MyDrive/Shared resources/Baron_pancreatic_islet.h5ad')
adata_train

AnnData object with n_obs × n_vars = 8569 × 20125
    obs: 'celltype'

## Preprocess the train data

In [ ]:
adata_train.obs_names_make_unique()
sc.pp.filter_cells(adata_train, min_genes=200)
sc.pp.filter_genes(adata_train, min_cells=3)
sc.pp.normalize_total(adata_train, target_sum=1e4)
sc.pp.log1p(adata_train)
#sc.pp.highly_variable_genes(adata_train, n_top_genes = 1000)
adata_train.raw = adata_train
#adata_train = adata_train[:, adata_train.var.highly_variable]
sc.pp.scale(adata_train, max_value=10)
adata_train.shape

(8569, 16359)

# Return logFC*expression train matrix


In [ ]:
def rank_mult_logFC_exp_train(adata_train, groupby_col, top_de_genes = 50):
  """
  This function find the top n upregulated and downregulated genes and returns the ranked logFC*expression matrix
  - We can do this step before or after dropout induction
  - We will select both upregulated and downregulated genes
  INPUT:
      data: anndata containing cell-gene expression matrix
      groupby_col: key of the observations grouping
      top_de_genes: number of top upregulated and downregulated de genes

  OUTPUT:
      adata: transformed anndata with rank ordered logFC*expression matrix
      adata.obs: transformed observations grouping
  """
  sc.tl.rank_genes_groups(adata_train, groupby = groupby_col, method='wilcoxon', use_raw = True)
  result = adata_train.uns['rank_genes_groups']
  groups = result['names'].dtype.names
  # Matrix sorted by logfoldchange
  detrain_dict = {}
  for group in groups:
      gene_rank_df = sc.get.rank_genes_groups_df(adata_train, group=group, pval_cutoff=0.05)
      print("Number of DE genes in {} = {}".format(group,len(gene_rank_df)))
      gene_rank_df.sort_values(by=['logfoldchanges'], inplace=True, ascending=False)
      if len(gene_rank_df) < 50:
          lfc_genes_df = gene_rank_df
      if len(gene_rank_df) >= 50:
          upregulated_genes = gene_rank_df.head(50)
          dnregulated_genes = gene_rank_df.tail(50)
          lfc_genes_df = pd.concat([upregulated_genes, dnregulated_genes], axis=0)
      detrain_dict[group] = dict(zip(lfc_genes_df['names'], lfc_genes_df['logfoldchanges']))

  # Take all the DE genes to create subset of genes in the main matrix 
  tot_gene_list = list(set([key for subdict in detrain_dict.values() for key in subdict.keys()]))
  print("Number of unique DE genes across all groups = {}".format(len(tot_gene_list)))
  
  # select only the subset of columns in the obs dataframe
  adata_sub = adata_train[:,tot_gene_list]

  # For each of the groups multiply the DE genes with the logFC with the expression
  sub_list = []
  for group in groups:
      gdata = adata_sub[adata_sub.obs[groupby_col] == group, :].to_df()
      for gene, factor in detrain_dict[group].items():
          gdata[gene] = gdata[gene]* abs(factor)
      gdata = gdata.assign(celltype=group)
      sub_list.append(gdata)

  del adata_sub, detrain_dict

  transformed_count  = pd.concat(sub_list, axis=0)
  transformed_group = transformed_count[[groupby_col]]
  # rank the values in each row
  df_ranked = transformed_count.drop(groupby_col, axis=1).rank(axis=1, method='min', ascending=False).astype(int)

  # create anndata for ranked tranformed matrix
  adata = sc.AnnData(df_ranked)
  adata.obs[groupby_col] = transformed_group
  
  del transformed_count, df_ranked
  return adata

In [ ]:
transform_adata_train = rank_mult_logFC_exp_train(adata_train, "celltype", 50)

Number of DE genes in acinar = 4897
Number of DE genes in activated_stellate = 3370
Number of DE genes in alpha = 4381
Number of DE genes in beta = 5051
Number of DE genes in delta = 2135
Number of DE genes in ductal = 5021
Number of DE genes in endothelial = 2561
Number of DE genes in epsilon = 34
Number of DE genes in gamma = 1118
Number of DE genes in macrophage = 1100
Number of DE genes in mast = 350
Number of DE genes in quiescent_stellate = 1635
Number of DE genes in schwann = 59
Number of DE genes in t_cell = 3
Number of unique DE genes across all groups = 983


In [ ]:
transform_adata_train

AnnData object with n_obs × n_vars = 8569 × 983
    obs: 'celltype'

In [ ]:
transform_adata_train.to_df()

index,CA12,CLGN,EHBP1,RHOA,LURAP1L,KCNH2,C1S,ERBB3,GREM2,PLTP,...,KLHDC8A,TPPP3,DLK1,COL12A1,ADIRF,FXYD3,BARX2,APOLD1,CD59,COL5A1
index,,,,,,,,,,,,,,,,,,,,,
human1_lib1.final_cell_0001,688,710,138,287,748,836,677,71,466,62,...,963,868,809,376,562,810,479,390,633,543
human1_lib1.final_cell_0002,708,733,805,603,111,850,698,718,498,746,...,963,877,818,408,120,819,512,421,132,577
human1_lib1.final_cell_0003,632,664,87,915,705,815,622,644,410,56,...,963,246,774,321,506,775,424,335,95,485
human1_lib1.final_cell_0004,120,704,786,813,744,828,669,688,471,716,...,963,854,801,379,564,802,483,393,671,545
human1_lib1.final_cell_0005,89,689,104,106,98,824,655,74,446,700,...,963,855,140,356,545,792,461,370,351,525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
human2_lib2.final_cell_0582,611,647,764,982,699,841,596,624,345,662,...,507,877,789,259,444,790,361,271,974,425
human2_lib2.final_cell_0590,604,641,760,38,694,838,589,617,338,656,...,499,873,785,254,436,786,354,266,975,417
human3_lib3.final_cell_0866,604,639,757,38,691,832,589,616,340,654,...,498,867,781,253,437,782,356,265,973,418


# Read Test data

In [ ]:
## Read test data
adata_test = sc.read('/content/gdrive/MyDrive/Shared resources/Segerstolpe_pancreatic_islet.h5ad')
adata_test

AnnData object with n_obs × n_vars = 2394 × 34363
    obs: 'celltype', 'tech'
    var: 'genename'

In [ ]:
adata_test.obs_names_make_unique()
sc.pp.filter_cells(adata_test, min_genes=200)
sc.pp.filter_genes(adata_test, min_cells=3)
sc.pp.normalize_total(adata_test, target_sum=1e4)
sc.pp.log1p(adata_test)
#sc.pp.highly_variable_genes(adata_train, n_top_genes = 1000)
adata_test.raw = adata_test
#adata_train = adata_train[:, adata_train.var.highly_variable]
sc.pp.scale(adata_test, max_value=10)
adata_test.shape

(2394, 21117)

# Return rank ordered test matrix

In [ ]:
def rank_exp_test(adata_test, gene_list):
  """
  This function find the common genes between train and test dataset. The output will be a rank ordered matrix.
  INPUT:
      data: anndata containing cell-gene expression matrix
      gene_list: gene sets from train data, adata_train.var_names

  OUTPUT:
      adata: transformed anndata with rank ordered  matrix
      adata.obs: transformed observations from original test anndata
  """
  # find the common genes between train and test dataset
  common_genes = gene_list.intersection(adata_test.var_names)
  print("Number of common genes of the reference and query sets = {}".format(len(common_genes)))
  
  # slice the test data based on common genes
  adata_test_aligned = adata_test[:, common_genes].copy()

  # rank order the test data
  df_ranked = adata_test_aligned.to_df().rank(axis=1, method='min', ascending=False).astype(int)

  # create anndata for ranked tranformed matrix
  adata = sc.AnnData(df_ranked)
  adata.obs = adata_test_aligned.obs

  del adata_test_aligned, df_ranked
  return adata
  


In [ ]:
transform_adata_test = rank_exp_test(adata_test, transform_adata_train.var_names)

Number of common genes of the reference and query sets = 963


In [ ]:
transform_adata_test

AnnData object with n_obs × n_vars = 2394 × 963
    obs: 'celltype', 'tech', 'n_genes'

In [ ]:
transform_adata_test.to_df()

index,CA12,CLGN,EHBP1,RHOA,LURAP1L,KCNH2,C1S,ERBB3,GREM2,PLTP,...,KLHDC8A,TPPP3,DLK1,COL12A1,ADIRF,FXYD3,BARX2,APOLD1,CD59,COL5A1
index,,,,,,,,,,,,,,,,,,,,,
AZ_A2,659,823,106,957,92,118,49,843,340,543,...,646,874,567,368,429,732,553,422,904,389
AZ_H5,643,803,863,400,844,868,666,825,320,524,...,631,860,547,350,410,916,534,404,786,372
AZ_G5,670,829,47,963,48,136,694,851,357,559,...,655,877,583,386,446,753,569,439,145,405
AZ_D8,706,857,873,105,890,74,732,877,399,598,...,59,901,618,429,488,180,607,482,187,450
AZ_D12,665,113,100,963,865,108,689,844,340,546,...,651,879,570,372,432,107,556,426,530,393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HP1504901_A22,591,748,816,77,62,821,613,772,305,478,...,577,64,500,332,1,880,486,375,81,53
HP1504901_M11,581,740,812,74,64,818,602,765,289,464,...,567,54,431,315,19,875,473,79,77,51
HP1504901_N21,593,753,113,110,103,824,619,776,323,481,...,582,72,502,344,32,878,488,384,70,58


---
# **Implementation of DL classifier**

---

train anndata = transform_adata_train

test anndata = transform_adata_test